+ More updates of D
+ Implicit weight clipping of D
+ Closer to vanilla WGAN per https://arxiv.org/pdf/1701.07875.pdf

In [1]:
%run 'Lib.ipynb'

In [ ]:
OUT_DIR = 'out_5'
import os
if not os.path.isdir(OUT_DIR): os.makedirs(OUT_DIR)

In [ ]:
from collections import defaultdict

import keras.backend as K
from keras.datasets import mnist
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.utils.generic_utils import Progbar

Using TensorFlow backend.


In [ ]:
Z_SIZE = 100

# batch and latent size taken from the paper
EPOCHS = 50
BATCH_SIZE = 100

# Adam parameters suggested in https://arxiv.org/abs/1511.06434
ADAM_LR = 0.0002
ADAM_BETA_1 = 0.5

# how many times to train D per each G(D) update
# 5 is suggested in https://arxiv.org/pdf/1701.07875.pdf
D_TRAIN_ITERATIONS = 5

# D weight clipping per https://arxiv.org/pdf/1701.07875.pdf
D_WEIGHTS_LOWER_BOUND = -0.01
D_WEIGHTS_UPPER_BOUND = 0.01

In [ ]:
np.random.seed(777)

In [ ]:
K.set_image_dim_ordering('tf')

In [ ]:
def wasserstein_distance(y_true, y_pred):
    return K.mean(y_true * y_pred)

In [ ]:
def create_D():
    input_image = Input(shape=(28, 28, 1), name='input_image')

    x = Conv2D(32, (3, 3), padding='same', name='conv_1')(input_image)
    x = LeakyReLU()(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Dropout(0.3)(x)

    x = Conv2D(64, (3, 3), padding='same', name='conv_2')(x)
    x = MaxPool2D(pool_size=1)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)

    x = Conv2D(128, (3, 3), padding='same', name='conv_3')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (3, 3), padding='same', name='coonv_4')(x)
    x = MaxPool2D(pool_size=1)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)

    features = Flatten()(x)

    output_is_fake = Dense(
        1, activation='linear', name='output_is_fake')(features)
    output_class = Dense(
        10, activation='softmax', name='output_class')(features)

    return Model(inputs=[input_image], outputs=[output_is_fake, output_class], name='D')

In [ ]:
def create_G(Z_SIZE=Z_SIZE):
    DICT_LEN = 10
    EMBEDDING_LEN = Z_SIZE

    # class#
    input_class = Input(shape=(1, ), dtype='int32', name='input_class')
    # encode class# to the same size as Z to use hadamard multiplication later on
    e = Embedding(
        DICT_LEN, EMBEDDING_LEN,
        embeddings_initializer='glorot_uniform')(input_class)
    embedded_class = Flatten(name='embedded_class')(e)

    # latent var
    input_z = Input(shape=(Z_SIZE, ), name='input_z')

    # hadamard product
    h = multiply([input_z, embedded_class], name='h')

    # cnn part
    x = Dense(1024)(h)
    x = LeakyReLU()(x)

    x = Dense(128 * 7 * 7)(x)
    x = LeakyReLU()(x)
    x = Reshape((7, 7, 128))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, (5, 5), padding='same')(x)
    x = LeakyReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (5, 5), padding='same')(x)
    x = LeakyReLU()(x)

    x = Conv2D(1, (2, 2), padding='same', activation='tanh', name='output_generated_image')(x)

    return Model(inputs=[input_z, input_class], outputs=x, name='G')

In [ ]:
D = create_D()

# SGD without momentum, see
# https://github.com/shekkizh/WassersteinGAN.tensorflow#observations
D.compile(
    optimizer=SGD(nesterov=False), 
    loss=[wasserstein_distance, 'sparse_categorical_crossentropy'])

In [ ]:
G = create_G()

G.compile(
    optimizer=Adam(lr=ADAM_LR, beta_1=ADAM_BETA_1), loss='binary_crossentropy')

In [ ]:
input_z = Input(shape=(Z_SIZE, ), name='input_z_')
input_class = Input(shape=(1, ),name='input_class_', dtype='int32')

In [ ]:
# create combined D(G) model
# D.trainable = False
output_is_fake, output_class = D(G(inputs=[input_z, input_class]))
combined = Model(inputs=[input_z, input_class], outputs=[output_is_fake, output_class])
combined.get_layer('D').trainable = False # freeze D in generator training faze

combined.compile(
    optimizer='RMSprop',
    loss=[wasserstein_distance, 'sparse_categorical_crossentropy']
)

In [ ]:
# try generating image

z = np.random.randn(100).reshape(1, 100)
img = G.predict([z, np.array([8])])
plt.imshow(img.reshape((28, 28)), cmap=plt.cm.gray)

In [ ]:
# get our mnist data, and force it to be of shape (..., 28, 28, 1) with
# range [-1., 1.]
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

X_test = (X_test.astype(np.float32) - 127.5) / 127.5
X_test = np.expand_dims(X_test, axis=3)

In [ ]:
plt.imshow(X_train[777].reshape((28, 28)), cmap=plt.cm.gray)

In [ ]:
train_num_samples, test_num_samples = X_train.shape[0], X_test.shape[0]

In [ ]:
train_history = defaultdict(list)
test_history = defaultdict(list)

In [ ]:
def save_generated_samples(epoch, G):

    # new batch of z's
    zz = np.random.normal(0., 1., (100, Z_SIZE))

    # new batch of classes [0..9]
    generated_classes = np.random.randint(0, 10, 100)

    generated_images = G.predict([zz, generated_classes.reshape(-1, 1)])

    rr = []
    for c in range(10):
        rr.append(np.concatenate(generated_images[c * 10:(1 + c) * 10]).reshape(280, 28))
    img = np.hstack(rr)

    plt.imsave(OUT_DIR + '/generated-e%d.png'%epoch, img, cmap=plt.cm.gray_r)

In [ ]:
for epoch in range(EPOCHS):

    print('\nEpoch {} / {}'.format(epoch + 1, EPOCHS))

    batches_num = int(train_num_samples / BATCH_SIZE)
    progress_bar = Progbar(target=batches_num)

    epoch_combined_losses = []
    epoch_D_losses = []

    for batch_i in range(batches_num):

        if len(epoch_D_losses) + len(epoch_combined_losses) > 1:
            progress_bar.update(
                batch_i,
                values=[
                        ('D_loss', np.mean(np.array(epoch_D_losses), axis=0)[0]),
                        ('D(G)_loss', np.mean(np.array(epoch_combined_losses),axis=0)[0])
                ]
            )
        else:
            progress_bar.update(batch_i)
            
        # 1: train D on real+generated images
        
        for d_it in range(D_TRAIN_ITERATIONS):

            # create batches of images and labels
            real_images = X_train[batch_i * BATCH_SIZE:(batch_i + 1) * BATCH_SIZE]
            real_images_classes = y_train[batch_i * BATCH_SIZE:(batch_i + 1) * BATCH_SIZE]

            # new batch of z's
            zz = np.random.normal(0., 1., (BATCH_SIZE, Z_SIZE))

            # new batch of classes [0..9]
            generated_classes = np.random.randint(0, 10, BATCH_SIZE)

            generated_images = G.predict([zz, generated_classes.reshape(-1, 1)])

            X = np.concatenate((real_images, generated_images))
            y_is_fake = np.array([-1] * BATCH_SIZE + [1] * BATCH_SIZE) # [is_fake=-1xBATCH_SIZE, 1xBATCH_SIZE]
            y_classes = np.concatenate((real_images_classes, generated_classes))

            # train 
            D_losses = D.train_on_batch(X, [y_is_fake, y_classes])
            epoch_D_losses.append(D_losses)

            # clip D weights

            for l in D.layers:
                weights = l.get_weights()
                weights = [np.clip(w, D_WEIGHTS_LOWER_BOUND, D_WEIGHTS_UPPER_BOUND) for w in weights]
                l.set_weights(weights)
        
        
        # 2: train D(G) (only G is trainable)
        
        # xxx (2 * D_TRAIN_ITERATIONS) so G is trained for the same number of images
        # xxx as D above (which is trained on real+generated)
        zz = np.random.normal(0., 1.,  (BATCH_SIZE, Z_SIZE))
        generated_classes = np.random.randint(0, 10, BATCH_SIZE)

        y_is_fake = -np.ones(2 * BATCH_SIZE) # all not-fake

        combined_losses = combined.train_on_batch(
            [zz, generated_classes.reshape((-1, 1))],
            [y_is_fake, generated_classes])
        
        epoch_combined_losses.append(combined_losses)
        
    print('\nD:', D.metrics_names)
    print(np.mean(np.array(epoch_D_losses), axis=0).tolist())
    
    print('D(G):', combined.metrics_names)
    print(np.mean(np.array(epoch_combined_losses), axis=0).tolist())
    
    save_generated_samples(epoch, G)

In [ ]:
# D

In [ ]:
# clamp_lower = -0.1
# clamp_upper = 0.1

# # Clip discriminator weights
# for l in D.layers:
#     weights = l.get_weights()
#     weights = [np.clip(w, clamp_lower, clamp_upper) for w in weights]
#     l.set_weights(weights)